In [5]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [2]:
import torch


### Initilize the connection with ElasticSearch

We already know that in open-domain Q&A, we typically design a model architecture that contains a data source, retriever, and reader/generator.

The first of these components is our document store. The two most popular stores are Elasticsearch and FAISS. FAISS is better but requires linux system

In [11]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

# initialize FAISS
document_store = ElasticsearchDocumentStore(
    host = 'localhost',
    username='', password='', 
    index='squad_docs'
)

08/18/2023 12:35:17 - INFO - elasticsearch -   HEAD http://localhost:9200/squad_docs [status:200 request:0.010s]
08/18/2023 12:35:17 - INFO - elasticsearch -   GET http://localhost:9200/squad_docs [status:200 request:0.004s]
08/18/2023 12:35:17 - INFO - elasticsearch -   PUT http://localhost:9200/squad_docs/_mapping [status:200 request:0.022s]
08/18/2023 12:35:17 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.006s]


### Dense Passage Retriever (DPR)

 DPR is converting the question you have provided into an approximate context. 

 The job of the retriever is to filter through our document store for relevant chunks of information (the documents) and pass them to the reader/generator model.

 The job of the retriever is critical to our reader performance. Given a query, it must find the most relevant contexts.

 DPR is an alternative to the traditional TF-IDF and BM25 techniques for passage retrieval. 
 DPR works by using two unique BERT encoder models. One of those models - Eᴘ - encodes passages of text into an encoded passage vector (we store context vectors in our document store).

The other model - EQ - maps a question into an encoded question vector.



In [13]:
from haystack.retriever.dense import DensePassageRetriever
# initialize DPR model
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

In [14]:
document_store.update_embeddings(retriever=retriever)

08/18/2023 12:48:24 - INFO - elasticsearch -   POST http://localhost:9200/squad_docs/_count [status:200 request:0.131s]
08/18/2023 12:48:24 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for 1204 docs ...
08/18/2023 12:48:24 - INFO - elasticsearch -   POST http://localhost:9200/squad_docs/_search?scroll=1d&size=10000 [status:200 request:0.259s]
08/18/2023 12:48:24 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.020s]
08/18/2023 12:48:24 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.007s]
Creating Embeddings: 100%|██████████| 76/76 [04:25<00:00,  3.49s/ Batches]
08/18/2023 12:53:00 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:5.357s]
08/18/2023 12:53:06 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:4.817s]
08/18/2023 12:53:08 - INFO - elasticsearch -   POST http://localhost

### Reader/ Generator model 

The reader/generator model is the final model in our Q&A stack. We can either have a reader, which extracts an answer directly from the context. Or, a generator, which uses language generation to generate an answer from the context.

In [15]:
from haystack.reader.farm import FARMReader
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2', use_gpu=True)

08/18/2023 12:53:08 - INFO - farm.utils -   Using device: CUDA 
08/18/2023 12:53:08 - INFO - farm.utils -   Number of GPUs: 1
08/18/2023 12:53:08 - INFO - farm.utils -   Distributed Training: False
08/18/2023 12:53:08 - INFO - farm.utils -   Automatic Mixed Precision: None
08/18/2023 12:53:20 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
08/18/2023 12:53:20 - INFO - farm.utils -   Using device: CUDA 
08/18/2023 12:53:20 - INFO - farm.utils -   Number of GPUs: 1
08/18/2023 12:53:20 - INFO - farm.utils -   Distributed Training: False
08/18/2023 12:53:20 - INFO - farm.utils -   Automatic Mixed Precision: None
08/18/2023 12:53:20 - INFO - farm.infer -   Got ya 3 parallel workers to do inference ...
08/18/2023 12:53:20 - INFO - farm.infer -    0    0    0 
08/18/2023 12:53:20 - INFO - farm.infer -   /w\  /w\  /w\
08/18/2023 12:53:20 - INFO - farm.infer -   /'\  / \  /'\
08/18/2023 12:53:20 - INFO - farm.infer -       


### Reader retriever pipeline with Haystack

Haystack comes with a very convenient ExtractiveQAPipeline class which allows us to pass our reader and retriever to build an easy-to-use extractive Q&A pipeline:


In [16]:
from haystack.pipeline import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

### Running Qs

In [28]:
pipeline.run(query='What is love?', top_k_reader=3)


Creating Embeddings: 100%|██████████| 1/1 [00:00<00:00,  6.99 Batches/s]
08/18/2023 13:01:57 - INFO - elasticsearch -   POST http://localhost:9200/squad_docs/_search [status:200 request:0.024s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 12.05 Batches/s]


{'query': 'What is love?',
 'no_ans_gap': -11.4096097946167,
 'answers': [{'answer': 'non-human primates',
   'score': -4.089797496795654,
   'probability': 0.3749064741977998,
   'context': 'f their forested territories. Meanwhile, the relationship between non-human primates in the subsistence and symbolism of indigenous lowland South Amer',
   'offset_start': 66,
   'offset_end': 84,
   'offset_start_in_doc': 347,
   'offset_end_in_doc': 365,
   'document_id': 'bb040628-4c48-4ddf-92f2-ca03d0ab87e5',
   'meta': {}},
  {'answer': 'Hugo',
   'score': -6.5083818435668945,
   'probability': 0.3071350115399719,
   'context': 'Janet Gray and other supporters of the hypothesis suggest that the name huguenote would be roughly equivalent to little Hugos, or those who want Hugo.',
   'offset_start': 145,
   'offset_end': 149,
   'offset_start_in_doc': 587,
   'offset_end_in_doc': 591,
   'document_id': '6580fe1b-bbaa-4342-8c8d-e16735ddfd56',
   'meta': {}},
  {'answer': 'first body exerts a for